# Capstone Project

## Business Idea

The main objective of the project is to explore venue-based similarity between different cities of the world. So, that cities can be clustered as per similarity and can help the users to find similar cities for comfort of travelling or migrating.


## Data

The cities data is obtained from the 'Simple Maps' Database, and 'FourSquare API' is used to explore the venues in each city. 
Each city will be explored by FourSquare data and features will be extracted and K-Means Algorithm will be applied.

Data source: https://simplemaps.com/data/world-cities